Pada notebook ini akan membuat model CNN untuk mengklasifikasikan foto kamar yang rapi atau berantakan. 

**LOAD DATASET**

In [1]:
# mendefinisikan nama direktori untuk data latih dan data validasi.
train_dir = "C:/Users/wiart/Documents/Belajar Machine Learning/Messy Bedroom Image Classification/images/train"
validation_dir = "C:/Users/wiart/Documents/Belajar Machine Learning/Messy Bedroom Image Classification/images/val"



In [2]:
import os

os.listdir(train_dir)

['clean', 'messy']

**PREPROCESSING DATA**

Langkah selanjutnya, kita akan menerapkan ImageDataGenerator untuk data latih dan data validasi. ImageDataGenerator merupakan sebuah fungsi yang sangat berguna untuk mempersiapkan data latih dan data validasi. Dataset akan dilakukan proses augmentasi gambar, dimana 
sebuah teknik yang dapat digunakan untuk memperbanyak data latih dengan cara menduplikasi gambar yang telah ada dengan menambahkan variasi tertentu

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_dataGen = ImageDataGenerator(
                    rescale = 1./255,
                    rotation_range = 20,
                    horizontal_flip = True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_dataGen = ImageDataGenerator(
                    rescale = 1./255)

Selanjutnya, siapkan data latih dan validasi dari kumpulan data gambar yang di-load dalam memori melalui fungsi flow()

In [6]:
train_generator = train_dataGen.flow_from_directory(
    train_dir,
    target_size = (150,150), # mengubah resolusi gambar menjadi 150x150
    batch_size = 4,
    class_mode = 'binary') # karena merupakan masalah klasifikasi 2 kelas, maka gunakan class_mode = binary

validation_generator = test_dataGen.flow_from_directory(
    validation_dir,
    target_size = (150,150), # mengubah resolusi gambar menjadi 150x150
    batch_size = 4,
    class_mode = 'binary') # karena merupakan masalah klasifikasi 2 kelas, maka gunakan class_mode = binary

Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


**MEMBUAT MODEL CNN**

Setelah data siap, selanjutnya adalah membangun model Convolutional Neural Network (CNN). Pembuatan model CNN pada keras mirip dengan pembuatan model Multi Layer Perceptron (MLP). Perbedaannya terdapat pada empat lapis layer konvolusi dan max pooling. Pada model CNN, proses klasifikasi gambar hanya berfokus pada atribut-atribut unik yang membedakan tiap kategori. Sehingga, teknik ini dinilai lebih optimal dibandingkan hanya menggunakan model MLP yang membedakan tiap kategori dengan melihat keseluruhan piksel-piksel pada gambar.

In [7]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Usai membuat model, gunakan fungsi summary() untuk melihat summary dari arsitektur model yang telah kita buat.

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

Berdasarkan hasil summary di atas, model yang kita buat terdiri dari empat lapis Convolutional dan MaxPoling layer, sebuah flatten layer, serta dua buah dense layer. Ingatlah bahwa dense layer terakhir merupakan output layer. Pada kasus klasifikasi biner, output model merupakan angka tunggal antara 0 dan 1. Sehingga, kita set dense layer terakhir = 1. Sementara itu, kolom “Param #” berisi informasi mengenai jumlah parameter pada tiap layer.

Selanjutnya, kolom “Output Shape” berisi informasi ukuran output yang dihasilkan tiap layer. Jika diperhatikan, ukuran input gambar yang telah didefinisikan sebelumnya adalah sebesar (150, 150). Tapi pada convolutional layer pertama, setiap satu input gambar akan menghasilkan ukuran output (148, 148) sebanyak 32 gambar. Ukuran tersebut berkurang karena kita menggunakan filter dengan ukuran (3, 3) dengan jumlah filter sebanyak 32 filter. Sehingga, tiap satu input gambar akan menghasilkan 32 gambar baru dengan ukuran (148, 148). 

Kemudian, resolusi tiap gambar akan diperkecil dengan tetap mempertahankan informasi pada gambar menggunakan MaxPoling layer yang berukuran (2, 2). Hal ini  akan menghasilkan ukuran output gambar sebesar (74, 74). Nah, proses tersebut juga berlaku untuk Convolutional dan MaxPoling layer yang lain. 

Berikutnya, pada flatten layer. Output dari MaxPoling layer terakhir yang terdiri dari 512 gambar dengan ukuran (7, 7) akan diubah ke dalam bentuk array 1D (tensor 1D). Hal ini  akan menghasilkan output berukuran (25088). 

Nah, output tersebut kemudian masuk ke dalam dense layer pertama yang memiliki 512 neuron. Sehingga, akan menghasilkan output dengan ukuran (512). Selanjutnya, output ini akan masuk pada dense layer kedua yang memiliki 1 neuron sehingga akan menghasilkan output dengan ukuran (1). Output dari layer terakhir inilah yang digunakan sebagai hasil akhir model untuk kasus klasifikasi biner.

Setelah membuat arsitektur model CNN, tahap selanjutnya adalah melakukan compile model tersebut menggunakan fungsi compile(). Loss function yang digunakan pada kasus klasifikasi biner adalah "binary_crossentropy". Selain itu, optimizer yang digunakan  pada kasus ini adalah "Adam optimizer". Adam optimizer dipilih karena mudah diterapkan, lebih efisien secara komputasi dan kebutuhan memori yang lebih kecil.

In [9]:
model.compile(loss='binary_crossentropy',
                optimizer=tf.optimizers.Adam(),
                metrics=['accuracy'])

**MODEL FITTING**

Pada proses ini, kita memasukkan data latih pada jaringan Neural Network yang telah kita buat sebelumnya. 

Hal yang harus didefinisikan pada tahap ini adalah loss function dan optimizer. Kemudian, mulai proses pelatihan model dengan memanggil fungsi fit(). 

Dengan menggunakan ImageDataGenerator, parameter tidak perlu dimasukan gambar dan labelnya. Image data generator secara otomatis melabeli gambar sesuai dengan direktorinya. Sebagai contoh,  sebuah gambar yang terdapat di direktori clean, akan diberi label “clean” oleh ImageDataGenerator secara otomatis.

In [10]:
model.fit(
    train_generator,
    steps_per_epoch=25, # berapa batch yang akan dieksekusi pada setiap epoch
    epochs=20, # tambahan epochs jika akurasi model belum optimal
    validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
    validation_steps=5, # berapa batch yang akan dieksekusi pada setiap epoch
    verbose=2
)

Epoch 1/20
25/25 - 11s - loss: 0.7523 - accuracy: 0.5200 - val_loss: 0.6924 - val_accuracy: 0.5000 - 11s/epoch - 427ms/step
Epoch 2/20
25/25 - 9s - loss: 0.6952 - accuracy: 0.4900 - val_loss: 0.6917 - val_accuracy: 0.7000 - 9s/epoch - 360ms/step
Epoch 3/20
25/25 - 9s - loss: 0.6713 - accuracy: 0.6200 - val_loss: 0.6937 - val_accuracy: 0.5000 - 9s/epoch - 373ms/step
Epoch 4/20
25/25 - 9s - loss: 0.7211 - accuracy: 0.5400 - val_loss: 0.6942 - val_accuracy: 0.5000 - 9s/epoch - 366ms/step
Epoch 5/20
25/25 - 9s - loss: 0.7056 - accuracy: 0.5400 - val_loss: 0.6907 - val_accuracy: 0.5000 - 9s/epoch - 365ms/step
Epoch 6/20
25/25 - 9s - loss: 0.6958 - accuracy: 0.4700 - val_loss: 0.6931 - val_accuracy: 0.5000 - 9s/epoch - 375ms/step
Epoch 7/20
25/25 - 10s - loss: 0.6934 - accuracy: 0.4000 - val_loss: 0.6931 - val_accuracy: 0.5000 - 10s/epoch - 391ms/step
Epoch 8/20
25/25 - 10s - loss: 0.6934 - accuracy: 0.5400 - val_loss: 0.6929 - val_accuracy: 0.5000 - 10s/epoch - 411ms/step
Epoch 9/20
25/25 -